## Tkinter GUI
### NHL/MLB result lookup by date and team

In [4]:
import tkinter as tk
from tkinter import *
import numpy as np
import pandas as pd
import requests  
from datetime import date as dt
from splinter import Browser
from bs4 import BeautifulSoup as soup
from datetime import date, timedelta
from webdriver_manager.chrome import ChromeDriverManager
import time
import re 

In [5]:
root = tk.Tk()
root.title("NHL/MLB Result Lookup")
canvas = tk.Canvas(root, height= 700, width= 800)
canvas.pack()
photo1 = PhotoImage(file="giphy.gif")
frame = tk.Frame(root, bg="white")
frame.place(relx=0.1, rely=0.1, relwidth=0.8, relheight=0.46) 

In [6]:
image = tk.Label(frame, image=photo1)
image.place(relx=.3,rely=0)

In [7]:
lower_frame = tk.Frame(root, bg="white", bd=10)
lower_frame.place(relx=0.1,rely=0.59, relwidth=.8, relheight=.2)
label1 = tk.Label(lower_frame)
label1.pack()

### Dropdown items

In [8]:
##DD/MM/YYYY

days = []
i=1
while i < 10:
    num = "0"+str(i)
    days.append(num)
    i = i +1
while i <32:
    days.append(str(i))
    i = i + 1
    
months = []
i = 1
while i <10:
    num = "0"+str(i)
    months.append(num)
    i = i+1
while i <13:
    months.append(str(i))
    i = i + 1

years = [*range(2022,1900,-1)]

### MLB team list


In [9]:
##MLB team list
mlb_list = ["Arizona Diamondbacks","Atlanta Braves","Baltimore Orioles","Boston Red Sox","Chicago White Sox","Chicago Cubs","Cincinnati Reds","Cleveland Indians","Colorado Rockies","Detroit Tigers","Houston Astros","Kansas City Royals","Los Angeles Angels","Los Angeles Dodgers",
"Miami Marlins","Milwaukee Brewers","Minnesota Twins","New York Yankees","New York Mets","Oakland Athletics","Philadelphia Phillies","Pittsburgh Pirates","San Diego Padres","San Francisco Giants","Seattle Mariners","St. Louis Cardinals","Tampa Bay Rays","Texas Rangers","Toronto Blue Jays","Washington Nationals"]

### NHL team list


In [10]:
nhl_teams = pd.read_csv("nhlteams.csv", header=None)
nhl = []
for team in nhl_teams[0]:
    nhl.append(team)

### Date Dropdown boxes

In [11]:
##Days dropdown menu
tkvardays = StringVar(root)
tkvardays.set(days[0])
popupMenu = OptionMenu(frame, tkvardays, *days)
Label(frame, text="Day").place(relx=0 ,rely=0 , relwidth=.1 ,relheight=.1 )
popupMenu.place(relx=0,rely=.1, relwidth=.1 ,relheight=.1 )

##Months dropdown menu
tkvarmnth = StringVar(root)
tkvarmnth.set(months[0])
popupMenu = OptionMenu(frame, tkvarmnth, *months)
Label(frame, text="Month").place(relx=.1 ,rely=0 , relwidth=.1 ,relheight=.1 )
popupMenu.place(relx=.1,rely=.1, relwidth=.1 ,relheight=.1 )

##Years dropdown menu
tkvaryr = StringVar(root)
tkvaryr.set(years[0])
popupMenu = OptionMenu(frame, tkvaryr, *years)
Label(frame, text="Year").place(relx=.2 ,rely=0 , relwidth=.1 ,relheight=.1 )
popupMenu.place(relx=.2,rely=.1, relwidth=.1 ,relheight=.1 )

### Sport selection box -> creates new dropdown based on selection

In [12]:
##Team list function
tkvarteam = StringVar(root)
def selected(event):
    if tkvar.get() == "Hockey":
        print("hockey")
        
        tkvarteam.set(nhl[0])
        popupMenu = OptionMenu(frame, tkvarteam, *nhl, command=set_team)
        Label(frame, text="NHL teams").place(relx=.05 ,rely=.6 , relwidth=.2 ,relheight=.1 )
        popupMenu.place(relx= .05,rely=.7 , relwidth=.2 ,relheight=.1 )
        
    else:
        
        tkvarteam.set(mlb_list[0])
        popupMenu = OptionMenu(frame, tkvarteam, *mlb_list, command=set_team)
        Label(frame, text="Mlb Teams").place(relx=.05 ,rely=.6 , relwidth=.2 ,relheight=.1 )
        popupMenu.place(relx= .05,rely=.7 , relwidth=.2 ,relheight=.1 )

## Team setting function
def set_team(event):
    print(tkvarteam.get())
    
## Pick sport -> trigger team list dropdown
tkvar = StringVar(root)
sports = ["Baseball", "Hockey"]
tkvar.set(sports[0])
popupMenu = OptionMenu(frame, tkvar, *sports, command=selected)
Label(frame, text="Choose a sport").place(relx=.05 ,rely=.3 , relwidth=.2 ,relheight=.1 )
popupMenu.place(relx=.05 ,rely=.4, relwidth=.2 ,relheight=.1 )

### NHL/MLB results function

In [13]:
def Did_They_win(day,month,year,sport,team):
    if sport == "Baseball":
        sport = "https://www.baseball-reference.com/boxes/?year="
    else:
        sport = "https://www.hockey-reference.com/boxscores/?year="

    executable_path = {'executable_path': ChromeDriverManager().install()}
    browser = Browser('chrome', **executable_path, headless=True)
    team = team+" "
    url = f'{sport}{year}&month={month}&day={day}'
    browser.visit(url)
    time.sleep(3)
    html = browser.html
    news_soup = soup(html, 'html.parser')
    yes = news_soup.find("div", {"class":"game_summaries"})
    yes1 = yes.find_all("tr", {"class":"winner"})
    yes2 = yes.find_all("tr", {"class":"loser"})
    browser.quit()
    losers = []
    for y in yes2:
        losers.append(y.text)
        wha = [re.sub('\s+', '', x) for x in losers]
        wha1 = [re.sub(r'(\w)([A-Z])', r'\1 \2', x) for x in wha]
        wha2 = [re.sub('([0-9])', r" \1", x) for x in wha1]
        wha3 = [re.sub('  Final| Final|Final| OT| SO', "", x) for x in wha2]
    winners = []
    for y in yes1:
        winners.append(y.text)
        win = [re.sub('\s+', '', x) for x in winners]
        win1 = [re.sub(r'(\w)([A-Z])', r'\1 \2', x) for x in win]
        win2 = [re.sub('([0-9])', r" \1", x) for x in win1]
        win3 = [re.sub('  Final| Final|Final| OT| SO', "", x) for x in win2]
    r1 = pd.DataFrame({"winner":[],"Goals":[]})
    i = 1
    for line in win3:
        team1 = line[:-1]
        goals = line[-1:]
        r1.loc[i] = [team1, goals]
        i = i+1
    r2 = pd.DataFrame({"loser":[],"Goals":[]})
    p = 1
    for line in wha3:
        team1 = line[:-1]
        goals = line[-1:]
        r2.loc[p] = [team1, goals]
        p = p+1
    
    
    kkkk = pd.DataFrame({"Winning Team":[],"W score":[], "Losing Team":[], "L score":[] })
    kkkk["Winning Team"] = r1["winner"]
    kkkk["Losing Team"] = r2["loser"]
    kkkk["W score"] = r1["Goals"]
    kkkk["L score"] = r2["Goals"]
   
    t = 1
    while t <= len(kkkk["Winning Team"]):
        if kkkk["Winning Team"][t] == team:
            lgoals = kkkk['W score'][t]
            ogoals = kkkk["L score"][t]
            lteam = kkkk["Losing Team"][t]
            label1['text'] = f"On {year}-{month}-{day} The {team}beat the {lteam} {lgoals} - {ogoals}"
            t = 2+ len(kkkk["Winning Team"])
        elif kkkk["Losing Team"][t] == team:
                lgoals = kkkk['W score'][t]                
                ogoals = kkkk["L score"][t]
                lteam = kkkk["Winning Team"][t]
                label1['text'] = f"On {year}-{month}-{day} The {team}lost to the {lteam} {lgoals} - {ogoals}"
                t = 2 + len(kkkk["Winning Team"])
        else:
            t = t+1
    if t == 1 + len(kkkk["Winning Team"]):
        label1['text'] = f"{team}did not play that day"
  

### Search button

In [14]:
button = tk.Button(frame, text="Click me, bitch", command= lambda: Did_They_win(tkvardays.get(),tkvarmnth.get(),tkvaryr.get(),tkvar.get(),tkvarteam.get()))
button.place(relx=.6,rely=.63)

### fin

In [ ]:
#create main loop
root.mainloop()

hockey
Toronto Maple Leafs




====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\Owner\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache


hockey
Chicago Blackhawks
Calgary Flames




====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\Owner\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Owner\AppData\Local\Programs\Python\Python310\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\Owner\AppData\Local\Temp\ipykernel_12368\1113547214.py", line 1, in <lambda>
    button = tk.Button(frame, text="Click me, bitch", command= lambda: Did_They_win(tkvardays.get(),tkvarmnth.get(),tkvaryr.get(),tkvar.get(),tkvarteam.get()))
  File "C:\Users\Owner\AppData\Local\Temp\ipykernel_12368\1840532890.py", line 35, in Did_They_win
    for line in win3:
UnboundLocalError: local variable 'win3' referenced before assignment


Minnesota Wild




====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\Owner\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Owner\AppData\Local\Programs\Python\Python310\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\Owner\AppData\Local\Temp\ipykernel_12368\1113547214.py", line 1, in <lambda>
    button = tk.Button(frame, text="Click me, bitch", command= lambda: Did_They_win(tkvardays.get(),tkvarmnth.get(),tkvaryr.get(),tkvar.get(),tkvarteam.get()))
  File "C:\Users\Owner\AppData\Local\Temp\ipykernel_12368\1840532890.py", line 35, in Did_They_win
    for line in win3:
UnboundLocalError: local variable 'win3' referenced before assignment


Calgary Flames




====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\Owner\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache
